In [ ]:
# Import required packages
import math, os, time

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import shap

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report, roc_auc_score, f1_score, recall_score, precision_score, SCORERS

from sklearn.ensemble import RandomForestClassifier
from xgboost.sklearn import XGBClassifier
from lightgbm.sklearn import LGBMClassifier
from sklearn.model_selection import GridSearchCV

pd.set_option('display.max_colwidth', None)
pd.options.display.max_rows = 999

for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

In [ ]:
try:
    del df
except:
    pass

df = pd.read_csv('/kaggle/input/characteristics-corona-patients/Characteristics_Corona_patients version 5 6-10-20 (1).csv')
df

In [ ]:
list(df.columns)

In [ ]:
# Dropping unused columns
df.drop(columns=['city',
                 'confirmed_date',
                 'infected_by',
                 'region',
                 'return_date',
                 'severity_illness_over_time',
                 'confirmed_date_D',
                 'deceased_date_D',
                 'released_date_D',
                 'return_date_D',
                 'date_onset_symptoms_D',
                 'return_date_until_date_onset_symptoms',
                 'date_onset_symptoms_until_confirmed_date',
                 'confirmed_date_until_released_date',
                 'confirmed_date_until_deceased_date',
                 'len_people_infected_by_patient',
                 'severity_illness_infectious_person'], inplace=True)
df

In [ ]:
# Dropping rows with values missing in both age + age_band
df.dropna(how='all',subset=['age','age_band'], inplace=True)
df.reset_index(drop=True,inplace=True)

print(df.notna().sum())
df

In [ ]:
# Dropping rows with values missing in both background_diseases + background_diseases_binary
df.dropna(how='all',subset=['background_diseases','background_diseases_binary'], inplace=True)
df.reset_index(drop=True,inplace=True)

print(df.notna().sum())
df

In [ ]:
# If all of the 3 columns are missing, drop the row (3 values required to find duration of disease)
df.dropna(how='all',subset=['date_onset_symptoms','deceased_date','released_date'],inplace=True)
df.reset_index(drop=True, inplace=True)

print(df.notna().sum())
df

In [ ]:
# Date onset required as reference point to determine duration (till release or death)
df.dropna(how='all',subset=['date_onset_symptoms'],inplace=True)
df.reset_index(drop=True, inplace=True)

print(df.notna().sum())
df

In [ ]:
df.dropna(how='all',subset=['sex'],inplace=True)
df.reset_index(drop=True, inplace=True)

print(df.notna().sum())
df

In [ ]:
# Filling NaNs for 'treatment' if 'released_date' is not null
def isNaN(string):
    return string != string

def fill_treatment(x, y):
    if isNaN(y):
        if isNaN(x) == False:
            return 'hospitalized'
        else:
            return y
    else:
        return y

df['treatment'] = df.apply(lambda x: fill_treatment(x['released_date'], x['treatment']), axis=1)
df

In [ ]:
# Dropping NaNs for 'treatment'
df.dropna(how='all',subset=['treatment'],inplace=True)
df.reset_index(drop=True, inplace=True)

print(df.notna().sum())
df

In [ ]:
# Fillna for missing 'smoking' values, Assumption: Most of undefined patients are non smokers (majority of indicated patients are non-smokers)
df['smoking'] = df['smoking'].fillna(0)

print(df.notna().sum())
df

In [ ]:
# Remove age, country
# Background_diseases --> for EDA (binary for model)
# Onset + deceased/released date [end date] --> Duration
# Hospitalized vs Home Isolation --> Differences in mortality [possibly 2 models]
# Treatment + Severity Illness --> Cured, Diseased, Treatment in Progress, Home Isolation
df.isna().sum()

In [ ]:
df[df['background_diseases_binary']==1]

1. age
2. BGD
3. country --> climate, (developing / developed) / (first, second, third world)
4. duration
5. status
6. gender
7. smoking
8. symptoms
9. 

In [ ]:
df[df['country']=='mexico']

In [ ]:
df.to_csv('covid19_filtered.csv', index=False)